In [36]:
###
### SORT ACCORDING TO COLUMN
###

import sys, csv ,operator
data = csv.reader(open('../INPUT_data/publishers.csv'))
sortedlist = sorted(data, key=operator.itemgetter(1))    # 1 specifies according to first column we want to sort
#now write the sorte result into new CSV file
with open("../INPUT_data/publishers_sorted.csv", "w") as f:
  fileWriter = csv.writer(f, delimiter=',')
  for row in sortedlist:
      fileWriter.writerow(row)



In [19]:
###
### DEDUPLICATE DATA PUBLISHERS
### Check manually at the end
###

import sys, csv ,operator
reader=csv.reader(open('../INPUT_data/publishers_sorted.csv', 'r'), delimiter=',')
writer=csv.writer(open('../INPUT_data/publishers_sorted_distinct.csv', 'w'), delimiter=',')
entries = set()

for row in reader:
   key = (row[0], row[1]) # instead of just the last name

   if key not in entries:
      writer.writerow(row)
      entries.add(key)





In [4]:
###
### DEDUPLICATE DATA PERIODICALS  (using Pandas)
### Check manually at the end
###


import pandas as pd
file_name = "../INPUT_data/periodicals.csv"
file_name_output = "../INPUT_data/periodicals_distinct.csv"

df = pd.read_csv(file_name, sep="\t or ,")

# Notes:
# - the `subset=None` means that every column is used 
#    to determine if two rows are different; to change that specify
#    the columns as an array
# - the `inplace=True` means that the data structure is changed and
#   the duplicate rows are gone  
df.drop_duplicates(subset=None, inplace=True)

# Write the results to a different file
df.to_csv(file_name_output)





/home/espadini/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # This is added back by InteractiveShellApp.init_path()


In [38]:
###
### DEDUPLICATE DATA AUTHORS  (using Pandas)
### Check manually at the end
###


import pandas as pd
file_name = "../INPUT_data/authors.csv"
file_name_output = "../INPUT_data/authors_distinct.csv"

df = pd.read_csv(file_name, sep="\t or ,")

# Notes:
# - the `subset=None` means that every column is used 
#    to determine if two rows are different; to change that specify
#    the columns as an array
# - the `inplace=True` means that the data structure is changed and
#   the duplicate rows are gone  
df.drop_duplicates(subset=None, inplace=True)

# Write the results to a different file
df.to_csv(file_name_output)

/home/espadini/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # This is added back by InteractiveShellApp.init_path()


In [46]:
###
### AUTHORS distinguish name and surname
###

import csv

f = open('../INPUT_data/authors_distinct.csv')
o = open('../INPUT_data/authors_distinct_surname_name2.csv', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

writer = csv.writer(o, quoting=csv.QUOTE_ALL)
for row in data[1:]:
    author = row[0]
    surname = author.partition(' ')[2] 
    name = author.partition(' ')[0]
    new_author = [surname, name]
    writer.writerow([new_author])

o.close()

In [1]:
###
### READ THE CSV (just to check)
###

import csv

f = open('../INPUT_data/publishers_sorted_distinct_manuallychecked.csv')
csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## TEST COSA C'È NELLE LINEE SPECIFICATE
## print(data[1:3])

for row in data[1:]:
    
    if (row[0] == ''):
        print('hello')


hello
hello
hello


<function TextIOWrapper.close>

In [17]:
###
### CREATE XML FROM CSV (PUBLISHERS)
###





from xml.etree import ElementTree as ET
import csv

f = open('../INPUT_data/publishers_sorted_distinct_manuallychecked.csv')
o = open('../OUTPUT_xml/publishers.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## TEST COSA C'È NELLE LINEE SPECIFICATE
## print(data[1:3])

## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')


## WRITE ITEMS TO FILE
for row in data[0:]:
    
    publisherLocation = row[0] 
    publisherName = row[1]
    word_list = row[1].replace("'", ' ') ## replace accent with space
    word_list = word_list.replace(",", '').split()  ## replace comma with nothing and split words
    words = '_'.join(word_list)     
    
    ## registering namespace
    NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
    NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
    ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
    ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)

    ## define elements with ns
    PublisherNS = ET.QName(NS_ROUD, "Publisher")
    labelNS = ET.QName(NS_KNORAIMPORT, "label")
    publisherHasLocationNS = ET.QName(NS_ROUD, "publisherHasLocation")
    publisherHasNameNS = ET.QName(NS_ROUD, "publisherHasName")

    ## create elements (as previously defined with ns)
    Publisher = ET.Element(PublisherNS, attrib={'id':words}) 
    label = ET.SubElement(Publisher, labelNS)
    label.text = "edi_"+publisherName
    
    ## if row is empty, don't create element, otherwise the import will fail (of course only for properties that are not mandatory)
    if (row[0] == ''):
        print()
    else:
        publisherHasLocation = ET.SubElement(Publisher, publisherHasLocationNS, attrib={'knoraType':'richtext_value'})
        publisherHasLocation.text = publisherLocation ## use variable defined with location of the publisher corresponding to first row
    
    publisherHasName = ET.SubElement(Publisher, publisherHasNameNS, attrib={'knoraType':'richtext_value'})
    publisherHasName.text = publisherName  ## use variable defined with name of the publisher corresponding to second row
    
    tree = ET.tostring(Publisher, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close


<function TextIOWrapper.close>

In [17]:
###
### CREATE XML FROM CSV (PERIODICALS)
###





from xml.etree import ElementTree as ET
import csv, re

f = open('../INPUT_data/periodicals_distinct.csv')
o = open('../OUTPUT_xml/periodicals.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## TEST COSA C'È NELLE LINEE SPECIFICATE
## print(data[1:3])

## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')


## WRITE ITEMS TO FILE
for row in data[0:]:
    
    periodicalTitle = row[0] 
    labelPeriodicalTitle = row[0].replace('"', '')  ## in label " not accepted
    word_list = re.split(', |\(|/', labelPeriodicalTitle) ## split at comma, parenthesis or slash and take the first part (take the first part is below), using labelPeriodicalTitle where quotes have been already deleted
    word_list = word_list[0].replace("'", ' ').replace(",", '').split()  ## replace accent with space and replace comma with nothing and split words
    words = '_'.join(word_list) 
    
    ## registering namespace
    NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
    NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
    ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
    ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)

    ## define elements with ns
    PeriodicalNS = ET.QName(NS_ROUD, "Periodical")
    labelNS = ET.QName(NS_KNORAIMPORT, "label")
    periodicalHasTitleNS = ET.QName(NS_ROUD, "periodicalHasTitle")

    ## create elements (as previously defined with ns)
    Periodical = ET.Element(PeriodicalNS, attrib={'id':words}) 
    label = ET.SubElement(Periodical, labelNS)
    label.text = "period_"+labelPeriodicalTitle
    periodicalHasTitle = ET.SubElement(Periodical, periodicalHasTitleNS, attrib={'knoraType':'richtext_value'})
    periodicalHasTitle.text = periodicalTitle  ## use variable defined with name of the publisher corresponding to second row
    
    tree = ET.tostring(Periodical, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close


<function TextIOWrapper.close>

In [45]:
###
### CREATE XML FROM CSV (AUTHORS)
###





from xml.etree import ElementTree as ET
import csv, re

f = open('../INPUT_data/authors_distinct_surname_name.csv')
o = open('../OUTPUT_xml/authors.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## TEST COSA C'È NELLE LINEE SPECIFICATE
## print(data[1:3])

## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')


## WRITE ITEMS TO FILE
for row in data[0:]:
    
    surname = row[1]
    name = row[0]
    #### for building id
    if len(surname.split()) > 1:  ## if long name and surname
        surnameid = '_'.join(re.split(' ', surname))
    else:
        surnameid = surname
    if len(name.split()) > 1:
        nameid = '_'.join(re.split(' ', name))
    else:
        nameid = name
    if (row[0] == ''):  ## if author has only surname
        authorid = (surnameid).replace("'", '').replace("(", '').replace(")", '') 
    else:
        authorid = (surnameid+'_'+nameid).replace("'", '').replace("(", '').replace(")", '') 
   

    
    ## registering namespace
    NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
    NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
    ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
    ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)

    ## define elements with ns
    AuthorNS = ET.QName(NS_ROUD, "Author")
    labelNS = ET.QName(NS_KNORAIMPORT, "label")
    AuthorHasFamilyNameNS = ET.QName(NS_ROUD, "authorHasFamilyName")
    AuthorHasGivenNameNS = ET.QName(NS_ROUD, "authorHasGivenName")

    ## create elements (as previously defined with ns)
    
    
    
    Author = ET.Element(AuthorNS, attrib={'id':authorid}) 
    label = ET.SubElement(Author, labelNS)
    if (row[0] == ''):  ## if author has surname and name
        label.text = "aut_"+surname
        authorHasFamilyName = ET.SubElement(Author, AuthorHasFamilyNameNS, attrib={'knoraType':'richtext_value'})
        authorHasFamilyName.text = surname  ## 
    else:
        label.text = "aut_"+surname+" "+name
        authorHasFamilyName = ET.SubElement(Author, AuthorHasFamilyNameNS, attrib={'knoraType':'richtext_value'})
        authorHasFamilyName.text = surname  ## 
        authorHasGivenName = ET.SubElement(Author, AuthorHasGivenNameNS, attrib={'knoraType':'richtext_value'})
        authorHasGivenName.text = name  ## 
    
    tree = ET.tostring(Author, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close


<function TextIOWrapper.close>

In [46]:
###
### CREATE XML FROM CSV (ARTICLES)
###



from xml.etree import ElementTree as ET
import csv, re

f = open('../INPUT_data/articles.csv')
o = open('../OUTPUT_xml/articles.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()


###################################
## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
###################################
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')



biblio_number_new = 1000  ## starting point for counting biblioid that should be added because are not in the original
    

###################################
## PREPARE CONTENT OF ELEMENTS AND ATTRIBUTES
###################################
for row in data[0:]:
    
    
    
    
    ## -----------------------> @id
    biblio_number_new += 1    
    if (row[0] != ''):
        Publicationid = 'biblio_'+row[0] ## @id
    else:
        Publicationid = 'biblio_'+str(biblio_number_new)   ## increasing number, just to give it an id. Starts from 1000
    
      
        
        
    
    ## -----------------------> hasPublicationType
    if ('Œuvre poétique' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-oeuvrePoetique'
    if ('Périodique' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-propos'
    ##if ('À propos de Roud' in row[1]):
      ##  HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-journal'
    if ('Traduction' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-traduction'
    if ('Photographie' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-photo'
    if ('Correspondance' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-correspondance'
    if ('À propos de Roud' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-surRoud'
    
    
    ## -----------------------> publicationHasTitle
    PublicationHasTitle = row[3]  
    
    
    ## -----------------------> isPublishedInPeriodical
    periodicalTitle = row[4].replace('"', '')  ## all this copied from transformation to periodical above
    word_list = re.split(', |\(|/', periodicalTitle) 
    word_list = word_list[0].replace("'", ' ').replace(",", '').split()  
    PeriodicalTarget = '_'.join(word_list) 
    
    
    ## -----------------------> publicationHasTitle
    HasCollaborators = row[6]  
    
    
    ## -----------------------> IsInPeriodicalIssue
    IsInPeriodicalIssueVolume = row[7]
    if (',' in row[7]):
        IsInPeriodicalIssue_split = re.split(', ',row[7])
        IsInPeriodicalIssue = IsInPeriodicalIssue_split[0]
        IsInPeriodicalVolume = IsInPeriodicalIssue_split[1].replace('-', ' et ')
    else:
        IsInPeriodicalVolumeOnly = row[7].replace('-', ' et ')
    
    
    ## -----------------------> publicationHasDate   
    Date = row[10]
    pattern = re.compile("\d\d\d\d-\d\d\d\d")
    if ('(' in Date):
        Datedmy_split = re.split('-', Date)  ## split in day, month, year
        Date1y = Datedmy_split[0]
        Date1m = Datedmy_split[1]
        Date12_split = re.split('\(', Date)  ## split in date1 and date2
        Date1 = Date12_split[0]
        Date2 = Date12_split[1].replace('(', '').replace(')', '')
        if len(Date1) > 7:             ## period includes year and month and days
            if len(Date2) > 3:         ## period includes year and month and days (month and days are different)
                PeriodMD = 'GREGORIAN:'+Date1+' CE:'+Date1y+Date2+' CE'  
            else:                      ## period includes year and month and days (only days are different)     
                PeriodD = 'GREGORIAN:'+Date1+' CE:'+Date1y+'-'+Date1m+Date2+' CE'  
        else:                          ## period includes year and month
            PeriodM = 'GREGORIAN:'+Date1+' CE:'+Date1y+Date2+' CE'
    else:
        if (pattern.match(Date)):
            DateY_split = re.split('-', Date)
            DateY1 = DateY_split[0]
            DateY2 = DateY_split[1]
            PeriodY = 'GREGORIAN:'+DateY1+' CE:'+DateY2+' CE'
        else:
            PublicationHasDate = 'GREGORIAN:'+Date+' CE'
    
    
    ## -----------------------> periodicalArticleIsInPages
    PeriodicalArticleIsInPages = row[11]  
    
    
    ## -----------------------> publicationHasInternalComment
    PublicationHasInternalComment = row[12]
    
    
    ## -----------------------> PublicationIsTranscribed
    PublicationIsTranscribed = row[13]
    
    
    ## -----------------------> PublicationIsTranscribed
    OriginalIsInCrlrArchive = row[16]
  
    
    ## -----------------------> publicationHasAuthor
    ArticleAuthor = row[2]
    if (',' in ArticleAuthor):
        ArticleAuthor_split = re.split(', ', ArticleAuthor)
        if (' ' in ArticleAuthor_split[0]):
            ArticlesSurname = ArticleAuthor_split[0].partition(' ')[0]
            ArticlesName = ArticleAuthor_split[0].partition(' ')[2]
            ArticlesSurnameSecond = ArticleAuthor_split[1].partition(' ')[0]
            ArticlesNameSecond = ArticleAuthor_split[1].partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0] and ArticlesName == row[1]):
                        AuthorTarget = row[2]
                    if (ArticlesSurnameSecond == row[0] and ArticlesNameSecond == row[1]):
                        AuthorTargetSecond = row[2]
        else:
            ArticlesSurname = ArticleAuthor_split[0]
            ArticlesSurnameSecond = ArticleAuthor_split[1].partition(' ')[0]
            ArticlesNameSecond = ArticleAuthor_split[1].partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0]):
                        AuthorTarget = row[2]
                    if (ArticlesSurnameSecond == row[0] and ArticlesNameSecond == row[1]):
                        AuthorTargetSecond = row[2]
    else:
        if (' ' in ArticleAuthor):
            ArticlesSurname = ArticleAuthor.partition(' ')[0]
            ArticlesName = ArticleAuthor.partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0] and ArticlesName == row[1]):
                        AuthorTargetSingle = row[2]
        else:
            ArticlesSurname = ArticleAuthor
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0]):
                        AuthorTargetSingle = row[2]  
        
    
    
    ## -----------------------> label
    if (ArticleAuthor != '' and PublicationHasTitle != '' and PeriodicalTarget != '' and Date != ''):
        PeriodicalArticleLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+PeriodicalTarget+' ___ '+Date
    else:
        if (ArticleAuthor != '' and PublicationHasTitle != '' and PeriodicalTarget != ''):
            PeriodicalArticleLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+PeriodicalTarget
        else:
            if (ArticleAuthor != '' and PublicationHasTitle != '' and Date != ''):
                PeriodicalArticleLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+Date
            else:
                if (ArticleAuthor != '' and PeriodicalTarget != '' and Date != ''):
                    PeriodicalArticleLabelComplete = "pub_"+ArticleAuthor+' ___ '+PeriodicalTarget+' ___ '+Date
                else:
                    if (PublicationHasTitle != '' and PeriodicalTarget != '' and Date != ''):
                        PeriodicalArticleLabelComplete = "pub_"+PublicationHasTitle+' ___ '+PeriodicalTarget+' ___ '+Date

    PeriodicalArticleLabel = re.sub(r"\(([^\)]+)\)", "", PeriodicalArticleLabelComplete)
    
    
    
                
                
    
    ###################################
    #### REGISTERING NAMESPACES
    ###################################
    NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
    NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
    ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
    ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)

    
    ###################################
    ## DEFINE ELEMENTS WITH NS
    ###################################
    PeriodicalArticleNS = ET.QName(NS_ROUD, "PeriodicalArticle")
    labelNS = ET.QName(NS_KNORAIMPORT, "label")
    hasPublicationTypeNS = ET.QName(NS_ROUD, "hasPublicationType")
    publicationHasAuthorNS = ET.QName(NS_ROUD, "publicationHasAuthor")
    AuthorNS = ET.QName(NS_ROUD, "Author")
    publicationHasTitleNS = ET.QName(NS_ROUD, "publicationHasTitle")
    isPublishedInPeriodicalNS = ET.QName(NS_ROUD, "isPublishedInPeriodical")
    PeriodicalNS = ET.QName(NS_ROUD, "Periodical")
    hasCollaboratorsNS = ET.QName(NS_ROUD, "hasCollaborators")
    isInPeriodicalIssueNS = ET.QName(NS_ROUD, "isInPeriodicalIssue")
    isInPeriodicalVolumeNS = ET.QName(NS_ROUD, "isInPeriodicalVolume")
    publicationHasDateNS = ET.QName(NS_ROUD, "publicationHasDate")
    periodicalArticleIsInPagesNS = ET.QName(NS_ROUD, "periodicalArticleIsInPages")
    publicationHasInternalCommentNS = ET.QName(NS_ROUD, "publicationHasInternalComment")
    publicationIsTranscribedNS = ET.QName(NS_ROUD, "publicationIsTranscribed")
    originalIsInCrlrArchiveNS = ET.QName(NS_ROUD, "originalIsInCrlrArchive")
    
    
    ###################################
    ## CREATE ELEMENTS AND ATTRIBUTES (AS PREVIOUSLY DEFINED WITH NS) AND ASSIGN THEM CONTENT
    ###################################
    PeriodicalArticle = ET.Element(PeriodicalArticleNS, attrib={'id':Publicationid}) 
    
    label = ET.SubElement(PeriodicalArticle, labelNS)
    label.text = PeriodicalArticleLabel
    
    
    ## -----------------------------> hasCollaborators
    if (HasCollaborators != ''):
        hasCollaborators = ET.SubElement(PeriodicalArticle, hasCollaboratorsNS, attrib={'knoraType':'richtext_value'})
        hasCollaborators.text = HasCollaborators
    
    
    ## -----------------------------> hasPublicationType
    hasPublicationType = ET.SubElement(PeriodicalArticle, hasPublicationTypeNS, attrib={'knoraType':'hlist_value'})
    hasPublicationType.text = HasPublicationType  ## use variable defined with name of the publisher corresponding to second row
    
    
    ## -----------------------------> isInPeriodicalVolume  and  isInPeriodicalIssue
    if (IsInPeriodicalIssueVolume != '' and IsInPeriodicalVolumeOnly == ''):
        isInPeriodicalIssue = ET.SubElement(PeriodicalArticle, isInPeriodicalIssueNS, attrib={'knoraType':'richtext_value'})
        isInPeriodicalIssue.text = IsInPeriodicalIssue
        isInPeriodicalVolume = ET.SubElement(PeriodicalArticle, isInPeriodicalVolumeNS, attrib={'knoraType':'richtext_value'})
        isInPeriodicalVolume.text = IsInPeriodicalVolume
    else:   
        if (IsInPeriodicalIssueVolume != ''):
            isInPeriodicalVolume = ET.SubElement(PeriodicalArticle, isInPeriodicalVolumeNS, attrib={'knoraType':'richtext_value'})
            isInPeriodicalVolume.text = IsInPeriodicalVolumeOnly
    
    
    ## -----------------------------> isPublishedInPeriodical
    isPublishedInPeriodical = ET.SubElement(PeriodicalArticle, isPublishedInPeriodicalNS)
    Periodical = ET.SubElement(isPublishedInPeriodical, PeriodicalNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':PeriodicalTarget})
    
    
    ## -----------------------------> OriginalIsInCrlrArchive
    if (OriginalIsInCrlrArchive != ''):
        originalIsInCrlrArchive = ET.SubElement(PeriodicalArticle, originalIsInCrlrArchiveNS, attrib={'knoraType':'richtext_value'})
        originalIsInCrlrArchive.text = OriginalIsInCrlrArchive
        
    
    ## -----------------------------> periodicalArticleIsInPages
    if (PeriodicalArticleIsInPages != ''):
        periodicalArticleIsInPages = ET.SubElement(PeriodicalArticle, periodicalArticleIsInPagesNS, attrib={'knoraType':'richtext_value'})
        periodicalArticleIsInPages.text = PeriodicalArticleIsInPages
        
    
    
    ## -----------------------------> publicationHasAuthor
    if (ArticleAuthor != ''):
        if (',') in ArticleAuthor:
            publicationHasAuthor = ET.SubElement(PeriodicalArticle, publicationHasAuthorNS)
            Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTarget})
            publicationHasAuthor = ET.SubElement(PeriodicalArticle, publicationHasAuthorNS)
            Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTargetSecond})
        else:
            publicationHasAuthor = ET.SubElement(PeriodicalArticle, publicationHasAuthorNS)
            Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTargetSingle})
    
    
    
    ## -----------------------------> publicationHasDate
    if (Date != ''):
        publicationHasDate = ET.SubElement(PeriodicalArticle, publicationHasDateNS, attrib={'knoraType':'date_value'})
    
    if ('(' in Date):
        if len(Date1) > 7:
            if len(Date2) > 3:   
                publicationHasDate.text = PeriodMD
            else:
                publicationHasDate.text = PeriodD
        else:
            publicationHasDate.text = PeriodM
    else:
        if (pattern.match(Date)):
            publicationHasDate.text = PeriodY
        else:
            if (Date != ''):
                publicationHasDate.text = PublicationHasDate
    
    
    ## -----------------------------> publicationHasInternalComment
    if (PublicationHasInternalComment != ''):
        publicationHasInternalComment = ET.SubElement(PeriodicalArticle, publicationHasInternalCommentNS, attrib={'knoraType':'richtext_value'})
        publicationHasInternalComment.text = PublicationHasInternalComment
        
    
    ## -----------------------------> publicationHasTitle
    publicationHasTitle = ET.SubElement(PeriodicalArticle, publicationHasTitleNS, attrib={'knoraType':'richtext_value'})
    publicationHasTitle.text = PublicationHasTitle
    
    
     ## -----------------------------> publicationIsTranscribed
    if (PublicationIsTranscribed != ''):
        publicationIsTranscribed = ET.SubElement(PeriodicalArticle, publicationIsTranscribedNS, attrib={'knoraType':'richtext_value'})
        publicationIsTranscribed.text = PublicationIsTranscribed
        
        
    
    
    tree = ET.tostring(PeriodicalArticle, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close


<function TextIOWrapper.close>

In [50]:
###
### CREATE XML FROM CSV (BOOKS)
###



from xml.etree import ElementTree as ET
import csv, re

f = open('../INPUT_data/books.csv')
o = open('../OUTPUT_xml/books.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()


## SI [0] id, [1] type, [2] author, [3] title, [6] collaborateurs, [8] placepub, [9] namepub, 
## SI [10] date, [11] pages, [12] comm interne, [13] Retranscrit, [15] digitized, [16] dans fonds CRLR  ##
## NO [4] title_pub, [5] illustré par, [7] numéro, [14] website interest  ##



###################################
## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
###################################
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')



biblio_number_new = 2000  ## starting point for counting biblioid that should be added because are not in the original
    

###################################
## PREPARE CONTENT OF ELEMENTS AND ATTRIBUTES
###################################
for row in data[0:]:
    
    
    
    
    ## -----------------------> @id
    biblio_number_new += 1    
    if (row[0] != ''):
        Publicationid = 'biblio_'+row[0] ## @id
    else:
        Publicationid = 'biblio_'+str(biblio_number_new)   ## increasing number, just to give it an id. Starts from 1000
    
      
        
        
    
    ## -----------------------> hasPublicationType
    if ('Œuvre poétique' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-oeuvrePoetique'
    if ('Périodique' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-propos'
    ##if ('À propos de Roud' in row[1]):
      ##  HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-journal'
    if ('Traduction' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-traduction'
    if ('Photographie' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-photo'
    if ('Correspondance' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-correspondance'
    if ('À propos de Roud' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-surRoud'
    
    
    ## -----------------------> publicationHasTitle
    PublicationHasTitle = row[3]  
    
    
    ## -----------------------> publicationHasCollaborators
    HasCollaborators = row[6]  
    
    
    ## -----------------------> hasPublisher (PublisherTarget)
    publisherName = row[9]
    word_list = publisherName.replace("'", ' ') ## replace accent with space
    word_list = word_list.replace(",", '').split()  ## replace comma with nothing and split words
    PublisherTarget = '_'.join(word_list)
    
    
    ## -----------------------> publicationHasDate   
    Date = row[10]
    pattern = re.compile("\d\d\d\d-\d\d\d\d")
    if ('(' in Date):
        Datedmy_split = re.split('-', Date)  ## split in day, month, year
        Date1y = Datedmy_split[0]
        Date1m = Datedmy_split[1]
        Date12_split = re.split('\(', Date)  ## split in date1 and date2
        Date1 = Date12_split[0]
        Date2 = Date12_split[1].replace('(', '').replace(')', '')
        if len(Date1) > 7:             ## period includes year and month and days
            if len(Date2) > 3:         ## period includes year and month and days (month and days are different)
                PeriodMD = 'GREGORIAN:'+Date1+' CE:'+Date1y+Date2+' CE'  
            else:                      ## period includes year and month and days (only days are different)     
                PeriodD = 'GREGORIAN:'+Date1+' CE:'+Date1y+'-'+Date1m+Date2+' CE'  
        else:                          ## period includes year and month
            PeriodM = 'GREGORIAN:'+Date1+' CE:'+Date1y+Date2+' CE'
    else:
        if (pattern.match(Date)):
            DateY_split = re.split('-', Date)
            DateY1 = DateY_split[0]
            DateY2 = DateY_split[1]
            PeriodY = 'GREGORIAN:'+DateY1+' CE:'+DateY2+' CE'
        else:
            PublicationHasDate = 'GREGORIAN:'+Date+' CE'
    
    
    ## -----------------------> bookHasSpecificPages
    BookHasSpecificPages = row[11]  
    
    
    ## -----------------------> publicationHasInternalComment
    PublicationHasInternalComment = row[12]
    
    
    ## -----------------------> PublicationIsTranscribed
    PublicationIsTranscribed = row[13]
    
    
    ## -----------------------> PublicationIsTranscribed
    OriginalIsInCrlrArchive = row[16]
  
    
    ## -----------------------> publicationHasAuthor
    ArticleAuthor = row[2]
    if (',' in ArticleAuthor):
        ArticleAuthor_split = re.split(', ', ArticleAuthor)
        if (' ' in ArticleAuthor_split[0]):
            ArticlesSurname = ArticleAuthor_split[0].partition(' ')[0]
            ArticlesName = ArticleAuthor_split[0].partition(' ')[2]
            ArticlesSurnameSecond = ArticleAuthor_split[1].partition(' ')[0]
            ArticlesNameSecond = ArticleAuthor_split[1].partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0] and ArticlesName == row[1]):
                        AuthorTarget = row[2]
                    if (ArticlesSurnameSecond == row[0] and ArticlesNameSecond == row[1]):
                        AuthorTargetSecond = row[2]
        else:
            ArticlesSurname = ArticleAuthor_split[0]
            ArticlesSurnameSecond = ArticleAuthor_split[1].partition(' ')[0]
            ArticlesNameSecond = ArticleAuthor_split[1].partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0]):
                        AuthorTarget = row[2]
                    if (ArticlesSurnameSecond == row[0] and ArticlesNameSecond == row[1]):
                        AuthorTargetSecond = row[2]
    else:
        if (' ' in ArticleAuthor):
            ArticlesSurname = ArticleAuthor.partition(' ')[0]
            ArticlesName = ArticleAuthor.partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0] and ArticlesName == row[1]):
                        AuthorTargetSingle = row[2]
        else:
            ArticlesSurname = ArticleAuthor
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0]):
                        AuthorTargetSingle = row[2]  
                        
                    
    ## -----------------------> label
    if (ArticleAuthor != '' and PublicationHasTitle != '' and Date != ''):
        BookLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+Date
    else:
        if (ArticleAuthor != '' and PublicationHasTitle != ''):
            BookLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle
        else:
            if (ArticleAuthor != '' and Date != ''):
                BookLabelComplete = "pub_"+ArticleAuthor+Date
            else:
                if (PublicationHasTitle != '' and Date != ''):
                    BookLabelComplete = "pub_"+PublicationHasTitle+' ___ '+Date

    BookLabel = re.sub(r"\(([^\)]+)\)", "", BookLabelComplete)

    
                
                
    
    ###################################
    #### REGISTERING NAMESPACES
    ###################################
    NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
    NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
    ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
    ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)

    
    ###################################
    ## DEFINE ELEMENTS WITH NS
    ###################################
    BookNS = ET.QName(NS_ROUD, "Book")
    labelNS = ET.QName(NS_KNORAIMPORT, "label")
    bookHasSpecificPagesNS = ET.QName(NS_ROUD, "bookHasSpecificPages")
    hasPublicationTypeNS = ET.QName(NS_ROUD, "hasPublicationType")
    hasPublisherNS = ET.QName(NS_ROUD, "hasPublisher")
    PublisherNS = ET.QName(NS_ROUD, "Publisher")
    publicationHasAuthorNS = ET.QName(NS_ROUD, "publicationHasAuthor")
    AuthorNS = ET.QName(NS_ROUD, "Author")
    publicationHasTitleNS = ET.QName(NS_ROUD, "publicationHasTitle")
    hasCollaboratorsNS = ET.QName(NS_ROUD, "hasCollaborators")
    publicationHasDateNS = ET.QName(NS_ROUD, "publicationHasDate")
    publicationHasInternalCommentNS = ET.QName(NS_ROUD, "publicationHasInternalComment")
    publicationIsTranscribedNS = ET.QName(NS_ROUD, "publicationIsTranscribed")
    originalIsInCrlrArchiveNS = ET.QName(NS_ROUD, "originalIsInCrlrArchive")
    
    
    ###################################
    ## CREATE ELEMENTS AND ATTRIBUTES (AS PREVIOUSLY DEFINED WITH NS) AND ASSIGN THEM CONTENT
    ###################################
    
    Book = ET.Element(BookNS, attrib={'id':Publicationid}) 
    
    label = ET.SubElement(Book, labelNS)
    label.text = BookLabel
    
    ## -----------------------------> bookHasSpecificPages
    if (BookHasSpecificPages!= ''):
        bookHasSpecificPages = ET.SubElement(Book, bookHasSpecificPagesNS, attrib={'knoraType':'richtext_value'})
        bookHasSpecificPages.text = BookHasSpecificPages
    
    
    ## -----------------------------> hasCollaborators
    if (HasCollaborators != ''):
        hasCollaborators = ET.SubElement(Book, hasCollaboratorsNS, attrib={'knoraType':'richtext_value'})
        hasCollaborators.text = HasCollaborators
    
    
    ## -----------------------------> hasPublicationType
    hasPublicationType = ET.SubElement(Book, hasPublicationTypeNS, attrib={'knoraType':'hlist_value'})
    hasPublicationType.text = HasPublicationType  ## use variable defined with name of the publisher corresponding to second row
    
    
    ## -----------------------------> hasPublisher
    if (publisherName != ''):
        hasPublisher = ET.SubElement(Book, hasPublisherNS)
        Publisher = ET.SubElement(hasPublisher, PublisherNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':PublisherTarget})
    
    
    
    ## -----------------------------> OriginalIsInCrlrArchive
    if (OriginalIsInCrlrArchive != ''):
        originalIsInCrlrArchive = ET.SubElement(Book, originalIsInCrlrArchiveNS, attrib={'knoraType':'richtext_value'})
        originalIsInCrlrArchive.text = OriginalIsInCrlrArchive
        
    
    ## -----------------------------> publicationHasAuthor
    if (',') in ArticleAuthor:
        publicationHasAuthor = ET.SubElement(Book, publicationHasAuthorNS)
        Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTarget})
        publicationHasAuthor = ET.SubElement(Book, publicationHasAuthorNS)
        Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTargetSecond})
    else:
        publicationHasAuthor = ET.SubElement(Book, publicationHasAuthorNS)
        Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTargetSingle})
    
    
    
    ## -----------------------------> publicationHasDate
    if (Date != ''):
        publicationHasDate = ET.SubElement(Book, publicationHasDateNS, attrib={'knoraType':'date_value'})
    
    if ('(' in Date):
        if len(Date1) > 7:
            if len(Date2) > 3:   
                publicationHasDate.text = PeriodMD
            else:
                publicationHasDate.text = PeriodD
        else:
            publicationHasDate.text = PeriodM
    else:
        if (pattern.match(Date)):
            publicationHasDate.text = PeriodY
        else:
            if (Date != ''):
                publicationHasDate.text = PublicationHasDate
    
    
    ## -----------------------------> publicationHasInternalComment
    if (PublicationHasInternalComment != ''):
        publicationHasInternalComment = ET.SubElement(Book, publicationHasInternalCommentNS, attrib={'knoraType':'richtext_value'})
        publicationHasInternalComment.text = PublicationHasInternalComment
        
    
    ## -----------------------------> publicationHasTitle
    publicationHasTitle = ET.SubElement(Book, publicationHasTitleNS, attrib={'knoraType':'richtext_value'})
    publicationHasTitle.text = PublicationHasTitle
    
    
     ## -----------------------------> publicationIsTranscribed
    if (PublicationIsTranscribed != ''):
        publicationIsTranscribed = ET.SubElement(Book, publicationIsTranscribedNS, attrib={'knoraType':'richtext_value'})
        publicationIsTranscribed.text = PublicationIsTranscribed
        
        
    
    
    tree = ET.tostring(Book, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close


<function TextIOWrapper.close>

In [15]:
###
### CREATE XML FROM CSV (BOOK SECTION)
###



from xml.etree import ElementTree as ET
import csv, re

f = open('../INPUT_data/booksections.csv')
o = open('../OUTPUT_xml/booksections.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()


## SI [0] id, [1] type, [2] author, [3] title, [4] title_pub, [6] collaborateurs, [7] volume, [8] placepub, [9] namepub, 
## SI [10] date, [11] pages, [12] comm interne, [13] Retranscrit, [15] digitized, [16] dans fonds CRLR  ##
## NO [5] illustré par, [14] website interest  ##



###################################
## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
###################################
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')



biblio_number_new = 3000  ## starting point for counting biblioid that should be added because are not in the original
    

###################################
## PREPARE CONTENT OF ELEMENTS AND ATTRIBUTES
###################################
for row in data[0:]:
    
    
    
    
    ## -----------------------> @id
    biblio_number_new += 1    
    if (row[0] != ''):
        Publicationid = 'biblio_'+row[0] ## @id
    else:
        Publicationid = 'biblio_'+str(biblio_number_new)   ## increasing number, just to give it an id. Starts from 1000
    
      
        
        
    
    ## -----------------------> hasPublicationType
    if ('Œuvre poétique' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-oeuvrePoetique'
    if ('Périodique' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-propos'
    ##if ('À propos de Roud' in row[1]):
      ##  HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-journal'
    if ('Traduction' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-traduction'
    if ('Photographie' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-photo'
    if ('Correspondance' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-correspondance'
    if ('À propos de Roud' in row[1]):
        HasPublicationType = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasPublicationType-surRoud'
    
    
    ## -----------------------> publicationHasTitle
    PublicationHasTitle = row[3]  
    
    
    ## -----------------------> bookSectionIsPartOf
    BookSectionIsPartOf = row[4]  
    
    
    ## -----------------------> publicationHasCollaborators
    HasCollaborators = row[6]  
    
    ## -----------------------> isInBookVolume
    IsInBookVolume = row[7]  
    
    
    ## -----------------------> hasPublisher (PublisherTarget)
    publisherName = row[9]
    word_list = publisherName.replace("'", ' ') ## replace accent with space
    word_list = word_list.replace(",", '').split()  ## replace comma with nothing and split words
    PublisherTarget = '_'.join(word_list)
    
    
    ## -----------------------> publicationHasDate   
    Date = row[10]
    pattern = re.compile("\d\d\d\d-\d\d\d\d")
    if ('(' in Date):
        Datedmy_split = re.split('-', Date)  ## split in day, month, year
        Date1y = Datedmy_split[0]
        Date1m = Datedmy_split[1]
        Date12_split = re.split('\(', Date)  ## split in date1 and date2
        Date1 = Date12_split[0]
        Date2 = Date12_split[1].replace('(', '').replace(')', '')
        if len(Date1) > 7:             ## period includes year and month and days
            if len(Date2) > 3:         ## period includes year and month and days (month and days are different)
                PeriodMD = 'GREGORIAN:'+Date1+' CE:'+Date1y+Date2+' CE'  
            else:                      ## period includes year and month and days (only days are different)     
                PeriodD = 'GREGORIAN:'+Date1+' CE:'+Date1y+'-'+Date1m+Date2+' CE'  
        else:                          ## period includes year and month
            PeriodM = 'GREGORIAN:'+Date1+' CE:'+Date1y+Date2+' CE'
    else:
        if (pattern.match(Date)):
            DateY_split = re.split('-', Date)
            DateY1 = DateY_split[0]
            DateY2 = DateY_split[1]
            PeriodY = 'GREGORIAN:'+DateY1+' CE:'+DateY2+' CE'
        else:
            PublicationHasDate = 'GREGORIAN:'+Date+' CE'
    
    
    ## -----------------------> bookSectionIsInPages
    BookSectionIsInPages = row[11]  
    
    
    ## -----------------------> publicationHasInternalComment
    PublicationHasInternalComment = row[12]
    
    
    ## -----------------------> PublicationIsTranscribed
    PublicationIsTranscribed = row[13]
    
    
    ## -----------------------> PublicationIsTranscribed
    OriginalIsInCrlrArchive = row[16]
  
    
    ## -----------------------> publicationHasAuthor
    ArticleAuthor = row[2]
    if (',' in ArticleAuthor):
        ArticleAuthor_split = re.split(', ', ArticleAuthor)
        if (' ' in ArticleAuthor_split[0]):
            ArticlesSurname = ArticleAuthor_split[0].partition(' ')[0]
            ArticlesName = ArticleAuthor_split[0].partition(' ')[2]
            ArticlesSurnameSecond = ArticleAuthor_split[1].partition(' ')[0]
            ArticlesNameSecond = ArticleAuthor_split[1].partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0] and ArticlesName == row[1]):
                        AuthorTarget = row[2]
                    if (ArticlesSurnameSecond == row[0] and ArticlesNameSecond == row[1]):
                        AuthorTargetSecond = row[2]
        else:
            ArticlesSurname = ArticleAuthor_split[0]
            ArticlesSurnameSecond = ArticleAuthor_split[1].partition(' ')[0]
            ArticlesNameSecond = ArticleAuthor_split[1].partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0]):
                        AuthorTarget = row[2]
                    if (ArticlesSurnameSecond == row[0] and ArticlesNameSecond == row[1]):
                        AuthorTargetSecond = row[2]
    else:
        if (' ' in ArticleAuthor):
            ArticlesSurname = ArticleAuthor.partition(' ')[0]
            ArticlesName = ArticleAuthor.partition(' ')[2]
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0] and ArticlesName == row[1]):
                        AuthorTargetSingle = row[2]
        else:
            ArticlesSurname = ArticleAuthor
            with open("../INPUT_data/authors_with_id.csv", 'r') as authors_with_id:   
                csv_authors = csv.reader(authors_with_id)
                ## AuthorTarget = [row[2] for row in csv_authors if ArticlesSurname in row[0]]  this creates list, while I want item
                for row in csv_authors:
                    if (ArticlesSurname == row[0]):
                        AuthorTargetSingle = row[2]  
                        
                    
    ## -----------------------> label
    if (ArticleAuthor != '' and PublicationHasTitle != '' and BookSectionIsPartOf != '' and Date != ''):
        BookSectionLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+BookSectionIsPartOf+' ___ '+Date
    else:
        if (ArticleAuthor != '' and PublicationHasTitle != '' and PeriodicalTarget != ''):
            BookSectionLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+BookSectionIsPartOf
        else:
            if (ArticleAuthor != '' and PublicationHasTitle != '' and Date != ''):
                BookSectionLabelComplete = "pub_"+ArticleAuthor+' ___ '+PublicationHasTitle+' ___ '+Date
            else:
                if (ArticleAuthor != '' and PeriodicalTarget != '' and Date != ''):
                    BookSectionLabelComplete = "pub_"+ArticleAuthor+' ___ '+BookSectionIsPartOf+' ___ '+Date
                else:
                    if (PublicationHasTitle != '' and PeriodicalTarget != '' and Date != ''):
                        BookSectionLabelComplete = "pub_"+PublicationHasTitle+' ___ '+BookSectionIsPartOf+' ___ '+Date

    BookSectionLabel = re.sub(r"\(([^\)]+)\)", "", BookSectionLabelComplete)

    
                
                
    
    ###################################
    #### REGISTERING NAMESPACES
    ###################################
    NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
    NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
    ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
    ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)

    
    ###################################
    ## DEFINE ELEMENTS WITH NS
    ###################################
    BookSectionNS = ET.QName(NS_ROUD, "BookSection")
    labelNS = ET.QName(NS_KNORAIMPORT, "label")
    bookSectionIsInPagesNS = ET.QName(NS_ROUD, "bookSectionIsInPages")
    bookSectionIsPartOfNS = ET.QName(NS_ROUD, "bookSectionIsPartOf")
    hasPublicationTypeNS = ET.QName(NS_ROUD, "hasPublicationType")
    isInBookVolumeNS = ET.QName(NS_ROUD, "isInBookVolume")
    bookSectionHasPublisherNS = ET.QName(NS_ROUD, "bookSectionHasPublisher")
    PublisherNS = ET.QName(NS_ROUD, "Publisher")
    publicationHasAuthorNS = ET.QName(NS_ROUD, "publicationHasAuthor")
    AuthorNS = ET.QName(NS_ROUD, "Author")
    publicationHasTitleNS = ET.QName(NS_ROUD, "publicationHasTitle")
    hasCollaboratorsNS = ET.QName(NS_ROUD, "hasCollaborators")
    publicationHasDateNS = ET.QName(NS_ROUD, "publicationHasDate")
    publicationHasInternalCommentNS = ET.QName(NS_ROUD, "publicationHasInternalComment")
    publicationIsTranscribedNS = ET.QName(NS_ROUD, "publicationIsTranscribed")
    originalIsInCrlrArchiveNS = ET.QName(NS_ROUD, "originalIsInCrlrArchive")
    
    
    ###################################
    ## CREATE ELEMENTS AND ATTRIBUTES (AS PREVIOUSLY DEFINED WITH NS) AND ASSIGN THEM CONTENT
    ###################################
    
    BookSection = ET.Element(BookSectionNS, attrib={'id':Publicationid}) 
    
    label = ET.SubElement(BookSection, labelNS)
    label.text = BookSectionLabel
    
    ## -----------------------------> bookSectionHasPublisher
    if (publisherName != ''):
        bookSectionHasPublisher = ET.SubElement(BookSection, bookSectionHasPublisherNS)
        Publisher = ET.SubElement(bookSectionHasPublisher, PublisherNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':PublisherTarget})
    
    
    ## -----------------------------> bookSectionIsInPages
    if (BookSectionIsInPages!= ''):
        bookSectionIsInPages = ET.SubElement(BookSection, bookSectionIsInPagesNS, attrib={'knoraType':'richtext_value'})
        bookSectionIsInPages.text = BookSectionIsInPages
        
    
        
    ## -----------------------------> BookSectionIsPartOf
    if (BookSectionIsPartOf!= ''):
        bookSectionIsPartOf = ET.SubElement(BookSection, bookSectionIsPartOfNS, attrib={'knoraType':'richtext_value'})
        bookSectionIsPartOf.text = BookSectionIsPartOf
    
    
    ## -----------------------------> hasCollaborators
    if (HasCollaborators != ''):
        hasCollaborators = ET.SubElement(BookSection, hasCollaboratorsNS, attrib={'knoraType':'richtext_value'})
        hasCollaborators.text = HasCollaborators
    
    
    ## -----------------------------> hasPublicationType
    hasPublicationType = ET.SubElement(BookSection, hasPublicationTypeNS, attrib={'knoraType':'hlist_value'})
    hasPublicationType.text = HasPublicationType  ## use variable defined with name of the publisher corresponding to second row
    
    
    ## -----------------------------> isInBookVolume
    if (IsInBookVolume != ''):
        isInBookVolume = ET.SubElement(BookSection, isInBookVolumeNS, attrib={'knoraType':'richtext_value'})
        isInBookVolume.text = IsInBookVolume
    
    
    ## -----------------------------> OriginalIsInCrlrArchive
    if (OriginalIsInCrlrArchive != ''):
        originalIsInCrlrArchive = ET.SubElement(BookSection, originalIsInCrlrArchiveNS, attrib={'knoraType':'richtext_value'})
        originalIsInCrlrArchive.text = OriginalIsInCrlrArchive
        
    
    ## -----------------------------> publicationHasAuthor
    if (',') in ArticleAuthor:
        publicationHasAuthor = ET.SubElement(BookSection, publicationHasAuthorNS)
        Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTarget})
        publicationHasAuthor = ET.SubElement(BookSection, publicationHasAuthorNS)
        Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTargetSecond})
    else:
        publicationHasAuthor = ET.SubElement(BookSection, publicationHasAuthorNS)
        Author = ET.SubElement(publicationHasAuthor, AuthorNS, attrib={'knoraType':'link_value', 'linkType':'ref', 'target':AuthorTargetSingle})
    
    
    
    ## -----------------------------> publicationHasDate
    if (Date != ''):
        publicationHasDate = ET.SubElement(BookSection, publicationHasDateNS, attrib={'knoraType':'date_value'})
    
    if ('(' in Date):
        if len(Date1) > 7:
            if len(Date2) > 3:   
                publicationHasDate.text = PeriodMD
            else:
                publicationHasDate.text = PeriodD
        else:
            publicationHasDate.text = PeriodM
    else:
        if (pattern.match(Date)):
            publicationHasDate.text = PeriodY
        else:
            if (Date != ''):
                publicationHasDate.text = PublicationHasDate
    
    
    ## -----------------------------> publicationHasInternalComment
    if (PublicationHasInternalComment != ''):
        publicationHasInternalComment = ET.SubElement(BookSection, publicationHasInternalCommentNS, attrib={'knoraType':'richtext_value'})
        publicationHasInternalComment.text = PublicationHasInternalComment
        
    
    ## -----------------------------> publicationHasTitle
    publicationHasTitle = ET.SubElement(BookSection, publicationHasTitleNS, attrib={'knoraType':'richtext_value'})
    publicationHasTitle.text = PublicationHasTitle
    
    
     ## -----------------------------> publicationIsTranscribed
    if (PublicationIsTranscribed != ''):
        publicationIsTranscribed = ET.SubElement(BookSection, publicationIsTranscribedNS, attrib={'knoraType':'richtext_value'})
        publicationIsTranscribed.text = PublicationIsTranscribed
        
        
    
    
    tree = ET.tostring(BookSection, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close

<function TextIOWrapper.close>

In [55]:
###
### CREATE XML FROM CSV (FICHES)
###



from xml.etree import ElementTree as ET
import csv, re

f = open('../INPUT_data/fiches_DONT_TOUCH.csv')   ## should not be manipulated, because when changing the content of certain cells go to the next cell and the entire row is erroneous
o = open('../OUTPUT_xml/fiches.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

##print(data[0:2])


## [0] id, [1] titre, [2] archive_id, [3] oldcote, [4] cote, [5] ensemble_id, [6] type_id, [7] annotation, [8] support_id, 
## [9] support_info, [10] instrument_id, [11] color_id, [12] other_tool, [13] statut_id, [14] dates, [15] datation, 
## [16] datationlist_id, [17] datationcomment, [18] publie, [19] biblio_id, [20] auteurtraduit_id, [21]alreadydigitized, 
## [22] numerise_info, [23] commentaire, [24] photocopy, [25] resp_id


###################################
## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
###################################
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')



###################################
#### REGISTERING NAMESPACES
###################################
NS_ROUD = "http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" 
NS_KNORAIMPORT = "http://api.knora.org/ontology/knoraXmlImport/v1#"
NS_KNORARICHTEXT = "http://example.com"
ET.register_namespace("p0112-roud-oeuvres", NS_ROUD)
ET.register_namespace("knoraXmlImport", NS_KNORAIMPORT)
ET.register_namespace("", NS_KNORATEXT)


###################################
## DEFINE ELEMENTS WITH NS
###################################
ManuscriptNS = ET.QName(NS_ROUD, "Manuscript")
labelNS = ET.QName(NS_KNORAIMPORT, "label")
hasAnnotationNS = ET.QName(NS_ROUD, "hasAnnotation")
hasDocumentTypeNS = ET.QName(NS_ROUD, "hasDocumentType")
hasGeneticStageNS = ET.QName(NS_ROUD, "hasGeneticStage")
hasOtherWritingToolNS = ET.QName(NS_ROUD, "hasOtherWritingTool")
richtextNS = ET.QName(NS_KNORARICHTEXT, "text")
hasPublicCommentNS = ET.QName(NS_ROUD, "hasPublicComment")
hasSupportInfoNS = ET.QName(NS_ROUD, "hasSupportInfo")
hasSupportTypeNS = ET.QName(NS_ROUD, "hasSupportType")
hasTranslatedAuthorNS = ET.QName(NS_ROUD, "hasTranslatedAuthor")
AuthorNS = ET.QName(NS_ROUD, "Author")
hasWritingColorNS = ET.QName(NS_ROUD, "hasWritingColor")
hasWritingToolNS = ET.QName(NS_ROUD, "hasWritingTool")
isPhotocopyNS = ET.QName(NS_ROUD, "isPhotocopy")
manuscriptHasDateNS = ET.QName(NS_ROUD, "manuscriptHasDate")
manuscriptHasDateEstablishedComputableNS = ET.QName(NS_ROUD, "manuscriptHasDateEstablishedComputable")
manuscriptHasDateEstablishedReadableNS = ET.QName(NS_ROUD, "manuscriptHasDateEstablishedReadable")
manuscriptHasEditorialSetNS = ET.QName(NS_ROUD, "manuscriptHasEditorialSet")
manuscriptHasInternalCommentNS = ET.QName(NS_ROUD, "manuscriptHasInternalComment")
manuscriptHasOldShelfmarkNS = ET.QName(NS_ROUD, "manuscriptHasOldShelfmark")
manuscriptHasPublishedReferenceNS = ET.QName(NS_ROUD, "manuscriptHasPublishedReference")
PublisherNS = ET.QName(NS_ROUD, "Publisher")
manuscriptHasShelfmarkNS = ET.QName(NS_ROUD, "manuscriptHasShelfmark")
manuscriptHasTitleNS = ET.QName(NS_ROUD, "manuscriptHasTitle")
manuscriptIsDigitizedNS = ET.QName(NS_ROUD, "manuscriptIsDigitized")
manuscriptIsInArchiveNS = ET.QName(NS_ROUD, "manuscriptIsInArchive")




###################################
## PREPARE CONTENT OF ELEMENTS AND ATTRIBUTES
###################################
for row in data[6:7]:
    
    
    ## -----------------------> @id + label
    ficheid = 'fiche'+row[0] 
    ficheLabel = ficheid
    
    
    ## -----------------------> hasAnnotation
    if (row[7] == 'oui'):
        hasAnnotation_content = 'true'
    else:
        hasAnnotation_content = 'false'    
    
    
    ## -----------------------> hasDocumentType
    documentType = row[6]
    if (documentType == '1'):
        hasDocumentType_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasDocumentType-manuscript'
    if (documentType == '2'):
        hasDocumentType_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasDocumentType-dactylo'
    if (documentType == '3'):
        hasDocumentType_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasDocumentType-imprime'
    
    
    ## -----------------------> hasGeneticStage
    geneticStage = row[13]
    if (geneticStage == '1'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-note'
    if (geneticStage == '2'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-plan'
    if (geneticStage == '3'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-brouillon'
    if (geneticStage == '4'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-miseAuNet'
    if (geneticStage == '5'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-manuscritDefinitif'
    if (geneticStage == '6'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-epreuvesCorriges'
    if (geneticStage == '7'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-originalCorrige'
    if (geneticStage == '8'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-verifier'
    if (geneticStage == '10'):
        hasGeneticStage_content = 'http://rdfh.ch/lists/0112/roud-oeuvres-flatlist-hasGeneticStage-liste'
    if (geneticStage == '11'):
        hasGeneticStage_content = 'NULL'
     
    
    ## -----------------------> hasOtherWritingTool
    otherWritingTool = row[12]
    if (otherWritingTool != ''):
        hasOtherWritingTool_content = otherWritingTool
        print(hasOtherWritingTool_content)
                
    
    
    
    ## LABEL DA AGGIUSTARE !!
    
    
    
    ###################################
    ## CREATE ELEMENTS AND ATTRIBUTES (AS PREVIOUSLY DEFINED WITH NS) AND ASSIGN THEM CONTENT
    ###################################
    
    
    
    
    Manuscript = ET.Element(ManuscriptNS, attrib={'id':ficheid}) 
    
    label = ET.SubElement(Manuscript, labelNS)
    label.text = ficheLabel
    
    
    ## -----------------------------> hasAnnotation
    hasAnnotation = ET.SubElement(Manuscript, hasAnnotationNS, attrib={'knoraType':'boolean_value'})
    hasAnnotation.text = hasAnnotation_content
    
    
    ## -----------------------------> hasDocumentType
    hasDocumentType = ET.SubElement(Manuscript, hasDocumentTypeNS, attrib={'knoraType':'hlist_value'})
    hasDocumentType.text = hasDocumentType_content
    
    
    ## -----------------------------> hasGeneticStage
    if (hasGeneticStage_content == 'NULL'):
        pass
    else:
        hasGeneticStage = ET.SubElement(Manuscript, hasGeneticStageNS, attrib={'knoraType':'hlist_value'})
        hasGeneticStage.text = hasGeneticStage_content
        
        
    ## -----------------------------> hasOtherWritingTool
    if (otherWritingTool != ''):
        hasOtherWritingTool = ET.SubElement(Manuscript, hasOtherWritingToolNS, attrib={'knoraType':'richtext_value', 'mapping_id':'http://rdfh.ch/standoff/mappings/StandardMapping'})
        richtext = ET.SubElement(hasOtherWritingTool, richtextNS, attrib={'xmlns':''})
        bla = ET.fromstring(hasOtherWritingTool_content)
        print(bla)
        richtext.append(bla)
    
   
        
    
    
    tree = ET.tostring(Manuscript, encoding="unicode")
    o.write('\n''\n'+ tree)

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close

<p>Stylo &agrave; bille, bleu Plume, noir Crayon, rouge Machine &agrave; &eacute;crire, rouge Premi&egrave;re lettre en rouge</p>


ParseError: undefined entity: line 1, column 9 (<string>)

In [43]:
###
### SCRAPS - SCARTI
###




### TEST SEMPLICE CREARE XML CON ELEMENTTREE

from xml.etree import ElementTree as ET

root = ET.Element("html")

head = ET.SubElement(root, "head")

title = ET.SubElement(head, "title")
title.text = "Page Title"

body = ET.SubElement(root, "body")
body.set("bgcolor", "#ffffff")

body.text = "Hello, World!"

tree = ET.tostring(root)
print(tree)

b'<html><head><title>Page Title</title></head><body bgcolor="#ffffff">Hello, World!</body></html>'


In [ ]:
###
### SCRAPS - SCARTI
###




### TENTATIVI CON LXML, LASCIAMO PERDERE PERCHÈ NON RIESCO AD AGGIUNGERE NAMESPACES

from lxml import etree

xhtml = etree.Element("{http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#}Publisher")
body = etree.SubElement(xhtml, "{http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#}label")
body.text = "Hello World"


root = etree.Element("root")
child2 = etree.SubElement(root, "child2")
child3 = etree.SubElement(root, "child3")
print(etree.tostring(root, pretty_print=True))


tag = etree.QName('http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#', 'p0112-roud-oeuvres')
root = etree.Element("{tag}root")
print(etree.tostring(root, pretty_print=True))



print(etree.tostring(xhtml, pretty_print=True))

In [ ]:
###
### SCRAPS - SCARTI
###




###
### LESS GOOD METHOD FOR CREATING XML FROM CSV
###




import csv

f = open('../INPUT_data/publishers_sorted_distinct_manuallychecked.csv')
o = open('../OUTPUT_xml/publishers_bahbah.xml', 'w')

csv_f = csv.reader(f)   
data = []

for row in csv_f: 
   data.append(row)
f.close()

## test cosa c'è nelle linee specificate
## print(data[1:3])

## FUNCTION THAT CREATES ITEM FROM CSV
def convert_row(row):
    ## ------->>>>>>> create id with first or first and last word of the publisher's name
    word_list = row[1].replace("'", ' ') ## replace accent with space
    word_list = word_list.replace(",", '').split()  ## replace comma with nothing and split words
    words = '_'.join(word_list)     
    
    return """
        <p0112-roud-oeuvres:Publisher id="edi_%s">
                <knoraXmlImport:label>edi_%s</knoraXmlImport:label>
                <p0112-roud-oeuvres:publisherHasLocation knoraType="richtext_value">%s</p0112-roud-oeuvres:publisherHasLocation>
                <p0112-roud-oeuvres:publisherHasName knoraType="richtext_value">%s</p0112-roud-oeuvres:publisherHasName>
            </p0112-roud-oeuvres:Publisher>
            """ % (words, row[1], row[0], row[1])


## WRITE DECLARATIONS AND BEGINNING OF THE XML FILE -adding more line breaks for readibility '+'\n'+' ???-
o.write('<?xml version="1.0" encoding="UTF-8"?>'+'\n'+'<knoraXmlImport:resources xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://api.knora.org/ontology/knoraXmlImport/v1# ../p0112-roud-oeuvres-xml-schemas/p0112-roud-oeuvres.xsd" xmlns="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#" xmlns:knoraXmlImport="http://api.knora.org/ontology/knoraXmlImport/v1#" xmlns:p0112-roud-oeuvres="http://api.knora.org/ontology/0112/roud-oeuvres/xml-import/v1#">'+'\n')

## WRITE ITEMS TO FILE, LOOP ON THE FUNCTION convert_row
o.write('\n'.join([convert_row(row) for row in data[1:]]))

## WRITE END OF THE XML FILE
with o:  ## this is to append the text, if just write o.write does not work here (why??)
    o.write('\n'+'</knoraXmlImport:resources>')

o.close


## TO HAVE SHORTER ID (but rules are more complicated -also considering that they will be reused after for linking- and anyway there will be exceptions, so ...)
##    word_list = row[1].replace("'", ' ') ## replace accent with space
##    word_list = word_list.replace(",", '').split()  ## replace comma with nothing and split words
##    if (word_list[0] == "La") or (word_list[0] == "L") or (word_list[0] == "Les") or (word_list[0] == "Le"): ## if the first word is ...
##        if (word_list[1] == word_list[-1]): ## if the second and the last words are equal
##            first_last_word = word_list[1]  ## take the second
##        else:
##            first_last_word = word_list[1]+'_'+word_list[-1]  ## otherwise take the second and the last
##    else:    ## if the first word is NOT ...
##        if (word_list[0] == word_list[-1]): ## if the first and the last words are equal
##            first_last_word = word_list[0]  ## take the first
##        else:
##            first_last_word = word_list[0]+'_'+word_list[-1]   ## otherwise take the first and the last 